# Data Formatter
----
In this notebook we'll process the raw CI log data into a format we can plug into the neural net. Specifically, we're going to take the various nested fields of the data, flatten it, map the dataset into a list of strings, and create a 35-dimensional vector representation of that list. We'll then pop those representations out into .npy files which the neural net notebook (neuralCI.ipynb) will be using. This notebook takes a little while to run (specifically, the dataset sampling section) but once it's run once, it shouldn't need to be run again.

## Imports and Spark Initialization

In [1]:
import os
import numpy as np

import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext,Row

spark = SparkSession.builder.master("local[*]").getOrCreate()

#stop weird parquet warns from cluttering logs
logger = spark._jvm.org.apache.log4j
logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )
logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )

## Load CI Data

In [2]:
cwd = os.getcwd()
data = spark.read.parquet("{}/rhci-moby.parquet".format(cwd))

## Flatten Data
The dataset comes with a very nested schema, which for our purposes is a hindrance. Let's flatten out the dataset. Annoyingly, there's no apparent built-in way to do this in Spark, so we have to write a bit of hacky code to do it. 

Spark lets you select and rename columns via:

`df.select(df[n_1].alias(a_1),...,df[n_n].alias(a_n))`

which means that we can select a nested field (say, a.b.c) and give it a top layer alias (a_b_c), it'll appear with other top layer fields. Therefore, if we can walk through the schema, find all subfields, give them top layer aliases, and then write some giant `select` statement to grab the 59 or so different fields, we can flatten our dataset.

However, writing a giant select statement by hand isn't exactly an elegant solution, so I wrote code that writes said giant select statement for us.

In [3]:
#create alias for schemas
def addAlias(string):
    labelString = string.replace(".","_")
    return "data['{}'].alias('{}')".format(string,labelString)

#recursively walk through schema and flatten, assign aliases
def flattenSchema(schema,label):
    global selector
    for field in schema:
        if isinstance(field.dataType,pyspark.sql.types.StructType):
            flattenSchema(field.dataType,label+field.name+".")
        else:   
            if field.name != None:
                selector.append(addAlias(label+field.name))

#using array of flat, aliased schemas, generate code that will flatten the dataframe
def createFlatFunc():
    global selector 
    selector = []
    flattenSchema(data.schema,"")
    flattener = ",".join(selector)
    return "flatData = data.select({})".format(flattener)

#execute the generated code
exec(createFlatFunc())

## Sample the Dataset
We don't need the full 220,000,000 points for our model. Let's shoot for ~100k points, 75k for training, 25k for testing. (Caching here takes a bit, but saves time later)

In [4]:
sampData = flatData.sample(False, 0.00045).cache()
sampData.count()

100326

## Stringify Data
The way we're going to attempt to model categorical data (i.e., almost all of the data) is by Word2Vec, so we need to turn each data point into a list of words. The simplest way to do this is to just cast each field of the data into string, but there's some unicode weirdness in the data, so we have to write function that's a bit more complex to do it.

In [5]:
def unicodeToString(row):
    ci_status = 1 if row[9]=="SUCCESS" else 0
    sentence = []
    for f,field in enumerate(row):
        if f == 9:
            continue
        elif isinstance(field,list):
            for item in field:
                sentence.append(str(item))
        else:
            if type(field)==type(u'unicode'):
                sentence.append(str(field.encode("utf8")))
            else:
                sentence.append(str(field))
    return Row(sentences=sentence,ci_status=ci_status)

strData = spark.createDataFrame(sampData.rdd.map(lambda x: unicodeToString(x)))

## Split Data into Training and Test Sets

In [6]:
train,valid = strData.randomSplit((.75,.25))
train.count()

75176

## Create Word2Vec Representation of Data
We want to make sure to train the word2Vec model on only the training data, obviously!

In [7]:
from pyspark.ml.feature import Word2Vec

word2vec = Word2Vec(vectorSize=35, minCount=0,inputCol='sentences',outputCol='result')
model = word2vec.fit(train)

trainRes = model.transform(train)
validRes = model.transform(valid)

tVectors  = trainRes.collect()
vVectors  = validRes.collect()

## Save and Write Vectors as Numpy Arrays

In [12]:
def numpify(data,label):
    status,sentences,vectors = zip(*data)
    X,Y = np.array(vectors),np.array(status)
        
    np.save(cwd+"/formattedData/"+label+"vectors.npy",X)
    np.save(cwd+"/formattedData/"+label+"status.npy",Y)
    return X,Y

numpify(tVectors,"t")
numpify(vVectors,"v")

[[ 0.01197636  0.10916476 -0.124254   ...,  0.13654004  0.04205332
  -0.0352273 ]
 [ 0.02029024  0.12202831 -0.12601285 ...,  0.12958692  0.04629133
  -0.04605272]
 [ 0.01799032  0.11729641 -0.12546606 ...,  0.13338823  0.04218564
  -0.04517513]
 ..., 
 [-0.06852289  0.10906283 -0.08026791 ...,  0.11507573 -0.02935873
  -0.06058261]
 [-0.10010055  0.18701809 -0.10264427 ...,  0.12897698  0.0058912
   0.01007479]
 [-0.09558322  0.17580825 -0.10442847 ...,  0.12370794  0.0159801
  -0.01374805]]
